# Forecast Agent Demo
### Combined Retail & Energy Forecasting with Agent Automation (Python 3.11)
This notebook demonstrates synthetic forecasting, evaluation, and agent-style automation for two domains:
- **Retail demand forecasting**
- **Energy load forecasting**

It includes:
- Synthetic data generation
- ARIMA / Prophet / XGBoost models
- Evaluation & drift visualization
- AI-Agent logic for automated decisions


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.arima.model import ARIMA
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

## Generate Synthetic Datasets

In [ ]:
# Retail demand data
dates = pd.date_range(start='2023-01-01', periods=300, freq='D')
sales = 200 + 10*np.sin(np.linspace(0, 20, 300)) + np.random.normal(0, 5, 300)
retail_df = pd.DataFrame({'ds': dates, 'y': sales})

# Energy load data
hours = pd.date_range(start='2023-01-01', periods=500, freq='H')
load = 500 + 50*np.sin(np.linspace(0, 50, 500)) + np.random.normal(0, 10, 500)
energy_df = pd.DataFrame({'ds': hours, 'y': load})

retail_df.head(), energy_df.head()

## Retail Forecasting — Prophet & ARIMA

In [ ]:
# Prophet model
prophet_model = Prophet()
prophet_model.fit(retail_df)
future = prophet_model.make_future_dataframe(periods=30)
forecast = prophet_model.predict(future)

# ARIMA model
arima_model = ARIMA(retail_df['y'], order=(5,1,0)).fit()
arima_forecast = arima_model.forecast(steps=30)

# Evaluation
actual = retail_df['y'][-30:].values
pred_prophet = forecast['yhat'][-30:].values
pred_arima = arima_forecast.values

mae_p = mean_absolute_error(actual, pred_prophet)
mae_a = mean_absolute_error(actual, pred_arima)

print(f"Prophet MAE: {mae_p:.2f}, ARIMA MAE: {mae_a:.2f}")

# Visualization
plt.figure(figsize=(10,5))
plt.plot(retail_df['ds'], retail_df['y'], label='Actual Sales')
plt.plot(forecast['ds'], forecast['yhat'], label='Prophet Forecast')
plt.title('Retail Demand Forecast (Prophet)')
plt.legend(); plt.show()

## Energy Forecasting — Prophet & XGBoost

In [ ]:
# Prophet model
energy_prophet = Prophet()
energy_prophet.fit(energy_df)
energy_future = energy_prophet.make_future_dataframe(periods=48, freq='H')
energy_fc = energy_prophet.predict(energy_future)

# XGBoost model
df_feat = energy_df.copy()
df_feat['hour'] = df_feat['ds'].dt.hour
X = df_feat[['hour']]
y = df_feat['y']
xgb_model = xgb.XGBRegressor(n_estimators=200, max_depth=4, learning_rate=0.1)
xgb_model.fit(X, y)
pred_xgb = xgb_model.predict(X)

# Evaluation
mae_e_p = mean_absolute_error(y[-48:], energy_fc['yhat'][-48:])
mae_e_x = mean_absolute_error(y[-48:], pred_xgb[-48:])
print(f"Energy Prophet MAE: {mae_e_p:.2f}, XGBoost MAE: {mae_e_x:.2f}")

# Visualization
plt.figure(figsize=(10,5))
plt.plot(energy_df['ds'], energy_df['y'], label='Actual Load')
plt.plot(energy_fc['ds'], energy_fc['yhat'], label='Prophet Forecast')
plt.title('Energy Load Forecast (Prophet)')
plt.legend(); plt.show()

## Agent Logic — Automated Evaluation & Action

In [ ]:
def agent_recommendation(domain, mae, threshold):
    if mae > threshold:
        return f"⚠️ {domain} model underperforming (MAE={mae:.2f}) — retrain recommended."
    else:
        return f"✅ {domain} model stable (MAE={mae:.2f})."

print(agent_recommendation('Retail Prophet', mae_p, 6))
print(agent_recommendation('Retail ARIMA', mae_a, 6))
print(agent_recommendation('Energy Prophet', mae_e_p, 12))
print(agent_recommendation('Energy XGBoost', mae_e_x, 12))

### End of Demo — Forecasting + AI Agent Simulation